# Tag-Based Search

In this notebook, we will introduce the concept of Tag-Based Search for long queries or multi-part queries.


### Python Imports


In [1]:
%load_ext autoreload
%autoreload 2


import sys
sys.path.append('..\\code')


import os
from dotenv import load_dotenv
load_dotenv()

from IPython.display import display, Markdown, HTML
from PIL import Image
from doc_utils import *


def show_img(img_path, width = None):
    if width is not None:
        display(HTML(f'<img src="{img_path}" width={width}>'))
    else:
        display(Image.open(img_path))


### Make sure we have the OpenAI Models information

We will need the GPT-4-Turbo and GPT-4-Vision models for this notebook.

When running the below cell, the values should reflect the OpenAI reource you have created in the `.env` file.

In [3]:
model_info = {
        'AZURE_OPENAI_RESOURCE': os.environ.get('AZURE_OPENAI_RESOURCE'),
        'AZURE_OPENAI_KEY': os.environ.get('AZURE_OPENAI_KEY'),
        'AZURE_OPENAI_MODEL_VISION': os.environ.get('AZURE_OPENAI_MODEL_VISION'),
        'AZURE_OPENAI_MODEL': os.environ.get('AZURE_OPENAI_MODEL'),
}

oai_client = AzureOpenAI(
    azure_endpoint = OPENAI_API_BASE, 
    api_key= AZURE_OPENAI_KEY,  
    api_version= AZURE_OPENAI_API_VERSION,
)

oai_emb_client = AzureOpenAI(
    azure_endpoint = AZURE_OPENAI_EMBEDDING_API_BASE, 
    api_key= AZURE_OPENAI_EMBEDDING_MODEL_RESOURCE_KEY,  
    api_version= AZURE_OPENAI_EMBEDDING_MODEL_API_VERSION,
)



### Make sure we have the AI Search information

When running the below cell, the values should reflect the AI Search reource you have created in the `.env` file, specifically, the `COG_SEARCH_ENDPOINT` and `COG_SEARCH_ADMIN_KEY` entries.

In [ ]:
os.environ.get('COG_SEARCH_ENDPOINT'), os.environ.get('COG_SEARCH_ADMIN_KEY')

### Create the Index

In [4]:
from utils.cogsearch_rest import *

index_name = 'tesla_facts'

fields = [
            {"name": "id", "type": "Edm.String", "key": True, "searchable": True, "filterable": True, "retrievable": True, "sortable": True},
            {"name": "vector", "type": "Collection(Edm.Single)", "searchable": True,"retrievable": True, "dimensions": 1536,"vectorSearchProfile": "my-vector-profile"},
            {"name": "tags", "type": "Edm.String","searchable": True, "filterable": False, "retrievable": True, "sortable": False, "facetable": False},
            {"name": "text", "type": "Edm.String","searchable": True, "filterable": False, "retrievable": True, "sortable": False, "facetable": False},
]

index = CogSearchRestAPI(index_name, fields=fields)

### Build the Index (optional)

In this section, we will build the vector index that will be used in the remainder of the notebook.

The dataset consists of 76 facts generated by GPT-4 around the Tesla Model S vehicle.

> **NOTE:**
> Please skip this cell if you have already built this index in the RAG_CE notebook.

In [ ]:
## IMPORTANT: Please skip this cell if you have already built this index in the RAG_CE notebook.

data = './sample_data/Tesla_Model_S.txt'

text = read_asset_file(data)[0]
facts = text.split('\n\n')

if index.get_index() is None:
    print(f"No index {index_name} detected, creating one ... ")
    index.create_index()

 metadatas = []

 for fact in facts: 
    metadata = {
        "text": fact, 
        "vector": get_embeddings(fact, client=oai_emb_client),
        "tags": generate_tag_list(fact, client=oai_client),
        "id": generate_uuid_from_string(fact)
    }

    metadatas.append(metadata)
    
    if len(metadatas) % 10 == 0:
        print(f"Processed {len(metadatas)} items")

# Save the data to a pickle file
save_to_pickle(metadatas, './sample_data/tesla_facts.pkl')

# Upload to the AI Search index
upload_output = index.upload_documents(metadatas)    

### Complex Queries

The below will be using the regular "one-shot" search, calculating only one embedding vector for the whole query.

In [36]:
query = """Given the Model S Plaid's acceleration speed and quarter mile completion time, the long-range version's driving range, the drag coefficient's role in aerodynamics, the integration of the battery pack for stability, the vehicle's safety rating, its towing capacity, and the presence of adaptive air suspension, what year did Tesla introduce a car that combines these performance, efficiency, safety, and utility features?
"""


results = call_ai_search(query, index_name, top=10)

for r in results:
    print(f"Distance Score: {bc.OKBLUE}{r['@search.score']:.5f}{bc.ENDC}\nText: {bc.OKGREEN}{r['text']}{bc.ENDC}\n\n")

Distance Score: 0.02790
Text: The Tesla Model S Plaid version boasts an extraordinary acceleration speed, sprinting from 0 to 60 mph in under 2 seconds precisely at 1.99 seconds, making it one of the fastest accelerating production cars available. Its power comes from a tri-motor setup that produces over 1,020 horsepower, enabling it to complete a quarter mile in just 9.23 seconds. 


Distance Score: 0.02803
Text: The Model S's performance version, the Plaid, can reach top speeds of up to 200 mph, making it not only one of the fastest accelerating electric cars but also one of the fastest in terms of top speed. This capability demonstrates Tesla's engineering prowess in electric vehicle performance.


Distance Score: 0.02509
Text: Tesla's Model S was the first electric vehicle to feature a dual motor setup, offering all-wheel drive (AWD) capabilities. This configuration not only improves traction and handling but also allows for better acceleration and efficiency compared to single mot

### Alternative: Tags-based Search

What if we took the same long queries and then generated tags out of them. Then each tag would be used for a search operation. The results of all the search operations (one search operation per tag), would then be 'interleaved' and then de-duplicated.

Let's first generate tags for the above query in the below cell.

In [59]:
tags = get_query_entities(query, approx_tag_limit=8)
tags

'Tesla Model S revolutionary vehicle technology, Tesla Model S system updates functionality, Tesla Model S feature minimizes operational costs, Tesla Model S design contributes to aerodynamic efficiency, Tesla Model S feature enhances passenger comfort, Tesla Model S capability for emergency braking, Tesla Model S impact on automotive industry, Tesla Model S technology advancements in electric vehicles'

And then in the below cell, we use tag-based search to get more varied and meaningful search results.

In [37]:
tag_results = aggregate_ai_search(query, index_name=index_name, top=2, approx_tag_limit=8)

for r in tag_results:
    print(f"Distance Score: {bc.OKBLUE}{r['@search.score']:.5f}{bc.ENDC}\nText: {bc.OKGREEN}{r['text']}{bc.ENDC}\n\n")


Search Intent Identification: Found 9 entities: ['Model S Plaid acceleration speed', 'Model S Plaid quarter mile completion time', 'Tesla long-range version driving range', 'drag coefficient role in Tesla aerodynamics', 'Tesla battery pack integration for stability', 'Tesla vehicle safety rating', 'Tesla towing capacity', 'Tesla adaptive air suspension', 'Year Tesla introduced car with performance efficiency safety utility features']
Distance Score: 0.02790
Text: The Tesla Model S Plaid version boasts an extraordinary acceleration speed, sprinting from 0 to 60 mph in under 2 seconds precisely at 1.99 seconds, making it one of the fastest accelerating production cars available. Its power comes from a tri-motor setup that produces over 1,020 horsepower, enabling it to complete a quarter mile in just 9.23 seconds. 


Distance Score: 0.01190
Text: The Model S's performance version, the Plaid, can reach top speeds of up to 200 mph, making it not only one of the fastest accelerating electri

In [54]:
query_template = """

## START OF CONTEXT
{context}
## END OF CONTEXT

Based on the Context above, please answer the below question. You **MUST** use the context to answer the question. If the answer is not in the context, it is ok to formulate a partial answer that fulfills part of the question. However, you **MUST NOT** fabricate facts or rely on your memory to answer. If the question requires calculations, make sure to break down your methodology step by step in the final answer, walk the user through your process, and show intermediate calculation results.

Question: {question}

"""

query = """
What makes the Tesla Model S a revolutionary vehicle in terms of technology, considering the system that updates its functionality, the feature that minimizes operational costs, the design that contributes to aerodynamic efficiency, the feature for enhancing passenger comfort, the capability for emergency braking, and its impact on the automotive industry?
"""


def get_answer(query, tag_based=False):

    if tag_based:
        results = aggregate_ai_search(query, index_name=index_name, top=2, approx_tag_limit=8)
    else:
        results = call_ai_search(query, index_name, top=10)

    context = '\n\n'.join([r['text'] for r in results])
    query_prompt = query_template.format(context=context, question=query)
    return ask_LLM(query_prompt, model_info=model_info)

answer = get_answer(query, tag_based=False)
Markdown(answer)

The Tesla Model S is a revolutionary vehicle in terms of technology for several reasons:

1. System that updates its functionality: The Model S features the ability to receive over-the-air software updates. This capability allows Tesla to remotely improve the car's performance, introduce new features, and fix bugs, ensuring that the vehicle continues to evolve and improve over time without the need for physical modifications.

2. Feature that minimizes operational costs: As an electric vehicle, the Model S minimizes operational costs by eliminating the need for gasoline and reducing maintenance requirements compared to traditional internal combustion engine vehicles. Additionally, the dual motor setup offers better efficiency, which can translate to cost savings in terms of energy consumption.

3. Design that contributes to aerodynamic efficiency: While the context does not explicitly mention aerodynamic design, Tesla vehicles are known for their focus on aerodynamics to reduce drag and improve range. The continuous updates to the Model S, including aesthetic refinements, suggest an ongoing commitment to design evolution, which likely includes aerodynamic improvements.

4. Feature for enhancing passenger comfort: The context does not provide specific details about features designed to enhance passenger comfort in the Model S. However, the large 17-inch touchscreen display that controls various functions like navigation and climate control contributes to a comfortable and convenient user experience.

5. Capability for emergency braking: The context does not mention emergency braking specifically for the Model S. However, given the introduction of advanced autonomous driving capabilities like Enhanced Autopilot in 2016, it is reasonable to infer that safety features such as emergency braking would be part of Tesla's suite of driver assistance technologies.

6. Impact on the automotive industry: The Tesla Model S has had a significant impact on the automotive industry by accelerating the adoption of electric vehicles worldwide. Its success has prompted traditional automakers to invest more heavily in electric technology, pushing the industry toward a more sustainable future. The Model S's influence has led to a shift in focus towards electric vehicles, with traditional automakers introducing competitive electric models into the market.

In summary, the Tesla Model S's technology, including its over-the-air updates, electric powertrain for reduced operational costs, design evolution for aerodynamics, touchscreen display for comfort and control, potential safety features like emergency braking, and its overall impact on the automotive industry, make it a revolutionary vehicle.

In [55]:
tag_answer = get_answer(query, tag_based=True)
Markdown(tag_answer)


Search Intent Identification: Found 8 entities: ['Tesla Model S revolutionary vehicle technology', 'Tesla Model S system updates functionality', 'Tesla Model S feature minimizes operational costs', 'Tesla Model S design contributes to aerodynamic efficiency', 'Tesla Model S feature enhances passenger comfort', 'Tesla Model S capability for emergency braking', 'Tesla Model S impact on automotive industry', 'Revolutionary vehicle in terms of technology']


The Tesla Model S is a revolutionary vehicle in terms of technology for several reasons:

1. **System that updates its functionality**: The Model S's ability to receive over-the-air software updates is a standout technological feature. This capability allows Tesla to remotely improve the car's performance, introduce new features, and fix bugs, ensuring that the vehicle continues to evolve and improve over time without the need for physical modifications.

2. **Feature that minimizes operational costs**: The electric powertrain of the Model S requires less maintenance than traditional internal combustion engines. With fewer moving parts, owners can expect lower service costs over the life of the vehicle. Additionally, the regenerative braking system recovers energy during braking and deceleration, which not only extends the vehicle's range but also reduces wear on the brake pads, further minimizing operational costs.

3. **Design that contributes to aerodynamic efficiency**: The Model S has a completely flat underbody, which contributes to its aerodynamic efficiency by reducing air resistance. This design helps to increase the range of the vehicle. Moreover, with a drag coefficient of just 0.208, the Model S is one of the most aerodynamically efficient cars in production, which also improves its range and speed.

4. **Feature for enhancing passenger comfort**: The unique glass roof of the Model S provides passengers with a panoramic view of the sky, enhancing the driving experience with an open, airy feel. This feature also includes UV and infrared protection to keep the cabin comfortable in various weather conditions. Additionally, the advanced suspension system can adjust the car's height in real-time based on driving conditions, ensuring a comfortable ride.

5. **Capability for emergency braking**: While the context does not explicitly mention emergency braking, it does refer to an advanced autopilot system that enables semi-autonomous driving capabilities. This system uses cameras, ultrasonic sensors, and radar to control steering, acceleration, and braking under certain conditions, which likely includes emergency braking scenarios to enhance driver safety and convenience.

6. **Impact on the automotive industry**: The success of the Tesla Model S has influenced the entire automotive industry to shift focus towards electric vehicles (EVs). Its pioneering features and achievements, such as breaking the 400-mile range barrier, have prompted traditional automakers to accelerate their own EV development and introduce competitive electric models into the market, pushing the industry toward a more sustainable future.

In summary, the Tesla Model S's technological advancements in software updates, maintenance efficiency, aerodynamic design, passenger comfort, safety features, and its broader impact on the automotive industry make it a revolutionary vehicle.

In [61]:
comparison = """ 

Answer 1:
## START OF ANSWER 1
{answer}
## END OF ANSWER 1

Answer 2:
## START OF ANSWER 2
{tag_answer}
## END OF ANSWER 2

Question: {question}

Compare the above Answer 1 and Answer 2, rate each of the answers in terms of completness verus the question with a score from 1 to 5, where 1 is the least complete and 5 is the most complete. Also, provide a brief explanation for your rating. Make sure to output the score and explanation for each answer.

Also, give a justification about which answer you would prefer and why.

"""


comparison_prompt = comparison.format(answer=answer, tag_answer=tag_answer, question=query)
comp_answer = ask_LLM(comparison_prompt, model_info=model_info)
Markdown(comp_answer)

Answer 1: Score - 4/5
Explanation: Answer 1 provides a comprehensive overview of the Tesla Model S's technological innovations and their impact on the automotive industry. It addresses each aspect mentioned in the question, such as the system that updates its functionality, the feature that minimizes operational costs, and the potential for emergency braking. However, it does not go into great detail about the design contributing to aerodynamic efficiency or specific features for enhancing passenger comfort, which are areas that could be expanded upon for completeness.

Answer 2: Score - 5/5
Explanation: Answer 2 offers a detailed response to the question, covering all the points raised. It explains the system that updates functionality, the feature that minimizes operational costs, the design that contributes to aerodynamic efficiency, the feature for enhancing passenger comfort, and the capability for emergency braking. Additionally, it provides specific examples, such as the flat underbody design and the panoramic glass roof, which add depth to the explanation. The answer also discusses the broader impact on the automotive industry, making it a very complete response.

Preferred Answer: I would prefer Answer 2 because it is more detailed and provides specific examples that directly address each point in the question. It gives a clearer picture of why the Tesla Model S is considered revolutionary in terms of technology and how it has influenced the automotive industry. The additional details about the aerodynamic design and passenger comfort features make the answer more informative and engaging.

### Real-Life Example

The below is a real-life query that was used to generate the contents of a powerpoint slide based on RAG search. The query is long and complex, and has multiple parts to it which need to be extracted and searched individually.

In [5]:
business_overview_query = """

###Instruction###

**You MUST work under the role of a Business Analyst assisting a team that is preparing a presentation for senior management. Your task is to provide an executive business overview of the requested company. Answer the following sections with the appropriate specifics, and where necessary, ask questions for further clarification.**

###Company Overview###

* Summarize the company's history, market entry, and operational milestones. Utilize bullet points for conciseness.
* Ensure your answer is unbiased and avoids relying on stereotypes.

###Key Financials###

* Extract and list key financial metrics such as:
     - EBITDA,
     - Profit Margins over the past five years,
     - Annual Revenue,
     - Compound Annual Growth Rate (CAGR).

###Sales Segmentation or Revenue Split###

* Provide sales segmentation by detailing customer demographics (such as age groups, geographic regions, and buying patterns), sales methods (including online, in-store, and direct sales), and distribution methods (like third-party or direct shipping).
* If full data is available, synthesize sales segmentation and revenue split data, pinpointing trends.
* In the absence of complete data, utilize available information to offer a reasoned approximation of these elements.

###Distribution Network###

* Present a brief outline of the distribution network, acknowledging the quantity and type of sales outlets.

###Self-evaluation Guidelines###

* Before delivering your response, please do a self-evaluation:
   - Ensure clarity and brevity by reviewing your response, aiming for succinctness without sacrificing completeness.
   - Rate the conciseness of your work on a scale of 1-10.
   - Confirm the accuracy of your answer internally.

###Example###

**Title**: Analysis of Minions Jeans Business Operations

**Company Overview**:
- Established European brand with global reach in 10 nations.
- Entered the Hungarian market in 1999, accounting for 12% of its global revenue.
- Operates on a perpetual license in Czech Republic as a prestigious casual wear brand.

**Key Financials**:
- Exhibited a robust CAGR of 25% from FY15 to FY19.
- EBITDA margins have experienced a downward trend in the past five years.

**Sales Segmentation**:
- Customer demographics show a strong preference among Men at 76%.
- Major sales contributions through Large Format Stores (LFS) at 22%.

**Distribution Network**:
- Total of 1,545 outlets, predominantly via LFS (320) and Multi-Brand Outlets/Franchisee owned (932), and others.

**Self-Evaluation**:
- Step by step review completed.
- Conciseness rated at 9.
- Confidence in accuracy: High.

"""

tags = get_query_entities(business_overview_query, approx_tag_limit=20)
tags


'Business Analyst role, Executive business overview creation, Presentation for senior management, Company history summarization, Market entry description, Operational milestones identification, Unbiased information provision, Stereotype avoidance, Key financial metrics extraction, EBITDA listing, Profit margins tracking over five years, Annual revenue listing, Compound Annual Growth Rate (CAGR) calculation, Sales segmentation detailing, Customer demographics analysis, Geographic regions sales analysis, Buying patterns identification, Sales methods categorization, Online sales data, In-store sales data, Direct sales information, Distribution methods outline, Third-party shipping data, Direct shipping information, Sales segmentation synthesis, Revenue split data analysis, Sales trends pinpointing, Distribution network outline, Sales outlets quantity specification, Sales outlets type specification, Clarity and brevity in responses, Conciseness self-rating, Accuracy confirmation in respons

### Plotting the Embeddings

Here we plot the embeddings of the original query as a whole, versus the embeddings of the different generated tags.

First, calculate the embeddings.

In [19]:
tags_arr = [s.strip() for s in tags.split(',')]
tags_embs = [get_embeddings(tag, client=oai_emb_client) for tag in tags_arr]
query_emb = get_embeddings(business_overview_query, client=oai_emb_client)

Now, generate the plots. 

In [21]:
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import hdbscan
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

embeddings = tags_embs + [query_emb]

# Perform dimensionality reduction using PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(embeddings)
pca_clusters = [1 for _ in tags_arr] + [2]  
metadatas = [{'tags': tag} for tag in tags_arr] + [{'tags': 'Business Overview Query'}]


# Convert clusters and results to a format suitable for Plotly
df_pca = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
df_pca['cluster'] = pca_clusters
df_pca['label'] = [f'Tags: {m["tags"][:200]}' for m in metadatas]  # Customize with desired metadata


# PCA Plot with Plotly
fig_pca = px.scatter(df_pca, x='PC1', y='PC2', color='cluster', hover_data=['label'])
fig_pca.update_layout(title='PCA of Embeddings', title_x=0.5)
fig_pca.show()